In [1]:
import os
os.chdir("..")
print(os.getcwd())

/workspace/rendering


In [2]:
import numpy as np
from pathlib import Path
import torch
from torch import nn

bindata = {}
for fp in sorted(Path("data/ManipNetBIN2").iterdir()):
    print(fp.stem)
    bindata[fp.stem] = np.fromfile(fp, dtype=np.float32)
for k, v in bindata.items():
    print(k, v.shape)

Decoder_b0
Decoder_w0
DenseRes0_b0
DenseRes0_w0
DenseRes1_b0
DenseRes1_w0
DenseRes2_b0
DenseRes2_w0
DenseRes3_b0
DenseRes3_w0
Encoder0_b0
Encoder0_w0
Encoder1_b0
Encoder1_w0
Encoder2_b0
Encoder2_w0
Xmean
Xstd
Ymean
Ystd
Decoder_b0 (214,)
Decoder_w0 (328704,)
DenseRes0_b0 (1536,)
DenseRes0_w0 (2359296,)
DenseRes1_b0 (1536,)
DenseRes1_w0 (2359296,)
DenseRes2_b0 (1536,)
DenseRes2_w0 (2359296,)
DenseRes3_b0 (1536,)
DenseRes3_w0 (2359296,)
Encoder0_b0 (512,)
Encoder0_w0 (98304,)
Encoder1_b0 (512,)
Encoder1_w0 (172032,)
Encoder2_b0 (512,)
Encoder2_w0 (932864,)
Xmean (2350,)
Xstd (2350,)
Ymean (214,)
Ystd (214,)


In [3]:
import torch.nn.functional as F

class Resblock(nn.Module):
    def __init__(self, in_vec, out_vec):
        super(Resblock, self).__init__()
        self.FC0 = nn.Linear(in_vec, out_vec)
        self.FC1 = nn.Linear(in_vec, out_vec)

    def __call__(self, H_zero):
        H_zero_ = F.relu(H_zero)
        H_one = self.FC0(H_zero_) + H_zero_
        H_one_ = F.relu(H_one)
        H_two = self.FC1(H_one_) + H_one_ + H_zero_
        return H_two



In [4]:
device = "cuda:0" if torch.cuda.is_available() else "cpu"
print("Using {} device".format(device))

Using cuda:0 device


In [5]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.Encoder0 = nn.Linear(192, 512)
        self.Encoder1 = nn.Linear(336, 512)
        self.Encoder2 = nn.Linear(1822, 512)
        self.DenseRes0 = Resblock(1536, 1536)
        self.DenseRes1 = Resblock(1536, 1536)
        self.Decoder = nn.Linear(1536, 214)

    def forward(self, pose, traj, sensor):
        pose_ = self.Encoder0(pose)
        traj_ = self.Encoder1(traj)
        sensor_ = self.Encoder2(sensor)
        out = torch.cat([pose_, traj_, sensor_], dim=-1)
        out = self.DenseRes0(out)
        out = self.DenseRes1(out)
        out = self.Decoder(out)
        return out

model  = NeuralNetwork().to(device)
print(device)
print(model)


cuda:0
NeuralNetwork(
  (Encoder0): Linear(in_features=192, out_features=512, bias=True)
  (Encoder1): Linear(in_features=336, out_features=512, bias=True)
  (Encoder2): Linear(in_features=1822, out_features=512, bias=True)
  (DenseRes0): Resblock(
    (FC0): Linear(in_features=1536, out_features=1536, bias=True)
    (FC1): Linear(in_features=1536, out_features=1536, bias=True)
  )
  (DenseRes1): Resblock(
    (FC0): Linear(in_features=1536, out_features=1536, bias=True)
    (FC1): Linear(in_features=1536, out_features=1536, bias=True)
  )
  (Decoder): Linear(in_features=1536, out_features=214, bias=True)
)


In [6]:
os.chdir("data/ManipNetBIN")
print(os.getcwd())

getbinname = {
    "Decoder.weight": "Decoder_w0",
    "Decoder.bias": "Decoder_b0",
    "Encoder0.weight": "Encoder0_w0",
    "Encoder0.bias": "Encoder0_b0",
    "Encoder1.weight": "Encoder1_w0",
    "Encoder1.bias": "Encoder1_b0",
    "Encoder2.weight": "Encoder2_w0",
    "Encoder2.bias": "Encoder2_b0",
    "DenseRes0.FC0.weight": "DenseRes0_w0",
    "DenseRes0.FC0.bias": "DenseRes0_b0",
    "DenseRes0.FC1.weight": "DenseRes1_w0",
    "DenseRes0.FC1.bias": "DenseRes1_b0",
    "DenseRes1.FC0.weight": "DenseRes2_w0",
    "DenseRes1.FC0.bias": "DenseRes2_b0",
    "DenseRes1.FC1.weight": "DenseRes3_w0",
    "DenseRes1.FC1.bias": "DenseRes3_b0",
}

with torch.no_grad():
    for name, param in model.named_parameters():
        fp = f"{getbinname[name]}.bin"
        savednp = np.fromfile(fp, dtype=np.float32)
        shape_ = param.shape
        param.copy_(torch.from_numpy(savednp.reshape(shape_, order="C")))

/workspace/rendering/data/ManipNetBIN


In [7]:
Xmean = np.fromfile("Xmean.bin", dtype=np.float32)
Ymean = np.fromfile("Ymean.bin", dtype=np.float32)
print(Xmean.shape, Xmean[-20:])
print(Ymean.shape, Ymean[-20:])

(2350,) [-0.61872  0.44557  0.17121 -0.65208  0.34153  0.36247  0.28213 -0.08111
 -0.28643 -0.33598  0.17554 -0.23346 -0.46493  0.07751 -0.10986 -0.52472
  0.10809  0.13183 -0.62187  0.22497]
(214,) [0.92892 0.77443 0.53388 0.6854  0.53704 0.5237  0.80279 0.66747 0.57714
 0.83595 0.76592 0.66795 0.87527 0.8128  0.73279 0.63066 0.63877 0.80745
 0.79434 0.76837]


In [8]:
loss_fn = nn.MSELoss()
model.to(device)

def test(bindata, model):
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        x, y = torch.from_numpy(bindata["Xmean"]), torch.from_numpy(bindata["Ymean"])
        pose = x[:192].to(device).unsqueeze(0)
        traj = x[192:528].to(device).unsqueeze(0)
        sensor = x[528:].to(device).unsqueeze(0)
        y = y.to(device).unsqueeze(0)
        pred = model(pose, traj, sensor)
        test_loss = loss_fn(pred, y)

    torch.set_printoptions(threshold = 10_000)
    print(f"Input: {x[:16]}")
    print(f"Actual: {y.squeeze(0)[:16]}")
    print(f"Predicted: {pred.squeeze(0)[:16]}")
    print(f"Loss: {test_loss}")

test(bindata, model)

Input: tensor([ 0.0000,  0.0000,  0.0000, -0.2251,  0.1177, -0.0623, -0.4226,  0.2596,
        -0.1927, -0.7172,  0.3679, -0.3507, -1.0464,  0.4562, -0.4728, -1.0769])
Actual: tensor([ 0.0000,  0.0000,  0.0000, -0.2251,  0.1177, -0.0623, -0.4226,  0.2596,
        -0.1927, -0.7172,  0.3679, -0.3507, -1.0464,  0.4563, -0.4728, -1.0769],
       device='cuda:0')
Predicted: tensor([ 0.0005,  0.0005,  0.0005,  0.0005,  0.0005,  0.0005, -0.1027,  0.0645,
        -0.0620, -0.3811,  0.1370,  0.0357, -0.4754, -0.0834,  0.0494,  0.0005],
       device='cuda:0')
Loss: 0.21744012832641602


In [9]:
os.chdir("../..")
print(os.getcwd())

/workspace/rendering


In [10]:
hov3_flame_number = 1000
filepaths_pkl = Path("data/HO3D_v3/train/").glob("*/meta/*.pkl")
filepaths_pkl_list = list(filepaths_pkl)

filepaths_bg = Path("data/HO3D_v3/train/").glob("*/rgb/*.jpg")
filepaths_bg_list = list(filepaths_bg)

object_number = 6
filepaths_obj = Path("data/models/").glob("*/textured_simple.obj")
filepaths_obj_list = list(filepaths_obj)

print(filepaths_pkl_list[hov3_flame_number])
print(filepaths_obj_list[object_number])

data/HO3D_v3/train/MDF11/meta/1016.pkl
data/models/008_pudding_box/textured_simple.obj


In [11]:
from utils.mano import ManoLayer
from utils.utils import load_ho_meta, get_obj_mesh, apply_transform_to_mesh, add_trimesh_to_fig, get_obj_mesh_print
import trimesh
import plotly.graph_objects as go
from scipy.spatial.transform import Rotation as R

In [12]:
obj_mesh = trimesh.load_mesh(filepaths_obj_list[object_number])

mano = ManoLayer()
anno = load_ho_meta(filepaths_pkl_list[hov3_flame_number])
mano_hand = mano(anno)

mesh = get_obj_mesh_print(anno)

obj_verts, obj_faces, obj_uv, obj_map = get_obj_mesh(anno)
obj_verts = apply_transform_to_mesh(obj_verts, anno)

mano_mesh = trimesh.Trimesh(vertices=mano_hand.vertices[0], face=mano.faces, )
obj_mesh = trimesh.Trimesh(vertices=obj_verts, faces=obj_faces)

from utils.sensor import AmbientSensor

amb = AmbientSensor()
amb.calc_ambient(mano_hand, obj_mesh)
print(amb.values.min(), amb.values.max())


/workspace/rendering/utils/utils.py:23: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  if v is None or v == 'None':


<trimesh.Trimesh(vertices.shape=(9174, 3), faces.shape=(15728, 3), name=`textured_simple.obj`)>
torch.Size([9174, 3])
torch.Size([15728, 3])
torch.Size([9174, 2])
torch.Size([4096, 4096, 3])
torch.Size([9174, 3]) torch.Size([3, 3]) torch.Size([3])
[[[ 0.    0.    0.    0.    0.    0.    0.    0.    0.    0.  ]
  [ 0.    0.    0.    0.    0.    0.    0.    0.    0.    0.  ]
  [ 0.    0.    0.    0.    0.    0.    0.    0.    0.    0.  ]
  [ 0.    0.    0.    0.    0.    0.    0.    0.    0.    0.  ]
  [ 0.    0.    0.    0.    0.    0.    0.    0.    0.    0.  ]
  [ 0.    0.    0.    0.    0.    0.    0.    0.    0.    0.  ]
  [ 0.    0.    0.    0.    0.    0.    0.    0.    0.    0.  ]
  [ 0.    0.    0.    0.    0.    0.    0.    0.    0.    0.  ]
  [ 0.    0.    0.    0.    0.    0.    0.    0.    0.    0.  ]
  [ 0.    0.    0.    0.    0.    0.    0.    0.    0.    0.  ]]

 [[-0.02 -0.02 -0.02 -0.02 -0.02 -0.02 -0.02 -0.02 -0.02 -0.02]
  [-0.02 -0.02 -0.02 -0.02 -0.02 -0.02 -0.02 -

In [13]:
fig = go.Figure()
add_trimesh_to_fig(fig, mano_mesh, opacity=0.8)
add_trimesh_to_fig(fig, obj_mesh, opacity=0.8)

fig.add_trace(
    go.Scatter3d(
        x = mano_hand.joints[:, 4, 0],
        y = mano_hand.joints[:, 4, 1],
        z = mano_hand.joints[:, 4, 2],
        mode = "markers",
        marker = dict(
            size=5,
            color="red",
            opacity = 1
        )
    )
)

# print("type_of_mano_hand", type(mano_hand.joints[:, 4, 0]))
# print("type_of_obj_mesh", type(amb.lattice[:, 0]))

fig.add_trace(
    go.Scatter3d(
        x = amb.lattice[:, 0],
        y = amb.lattice[:, 1],
        z = amb.lattice[:, 2],
        mode = "markers",
        marker = dict(
            size=2,
            color=amb.values,
            colorscale="Viridis",
            opacity=0.5
        )
    )
)

print(amb.values)
print(amb.values.shape)

tensor([0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 1.7556, 1.2212, 1.4470, 1.7889, 1.7611, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 1.4456, 1.0000, 1.0000, 1.2520, 1.4055, 1.6029,
        0.0000, 0.0000, 0.0000, 0.0000, 1.6976, 1.3112, 1.2881, 1.3499, 1.3629,
        1.5152, 1.7305, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 1.7162,
        1.7852, 1.8233, 1.9498, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 1.8636, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 1.4804, 